In [13]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/kaggle/code/core')
sys.path.append('/kaggle/input/my-flg-library/')
import flg_support as fls
import importlib
import numpy as np
import flg_diagnostics
import flg_numerics
import matplotlib.pyplot as plt
import flg_yolo
import flg_yolo2
import flg_model

In [14]:
# train_data = fls.load_all_train_data()
# model = fls.dill_load(fls.temp_dir + 'yolo_test.pickle')
# model.confidence_threshold = 0.1
# rr = model.infer(train_data[19:21])
# res = [r.labels for r in rr]

# print(rr[0].labels_unfiltered)
# print(rr[1].labels_unfiltered)
# print(res)

In [16]:
#importlib.reload(flg_yolo2)
#importlib.reload(flg_model)
train_data = fls.load_all_train_data()
model_orig = fls.dill_load(fls.temp_dir + 'yolo_test.pickle')
model_orig.step1Labels.confidence_threshold = 0.1

train_data = fls.load_all_train_data()
model = flg_model.TwoStepModel()
model.run_in_parallel = False
model.step1Labels.trained_model = model_orig.step1Labels.trained_model
model.step1Labels.confidence_threshold = 0.1
model.calibrate_step_2 = True
#model.step1Labels.infer(train_data[1])
model.train(train_data[1:150], train_data[216:230])

# train_data = fls.load_all_train_data()
# model = fls.dill_load(fls.temp_dir + 'yolo_test.pickle')

Clearing pytorch
(300, 959, 928) float16
Clearing cupy
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Processing tomogram tomo_556257 (1/1)
Processing 300 out of 300 slices (CONCENTRATION=1)


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4086: error: (-215:Assertion failed) func != 0 in function 'cv::hal::resize'


In [5]:
model.step2Output.threshold = 0.1
rr = model.infer(train_data[19:21])
res = [r.labels for r in rr]

print(rr[0].labels_unfiltered)
print(rr[1].labels_unfiltered)
print(res)

(300, 959, 928) uint8
Clearing cupy
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Processing tomogram tomo_081a2d (1/1)
Processing 300 out of 300 slices (CONCENTRATION=1)
[{'z': 155, 'y': 319, 'x': 644, 'confidence': 0.6162109375}]
Clearing pytorch
(500, 924, 956) uint8
Clearing cupy
Processing tomogram tomo_08446f (1/1)
Processing 500 out of 500 slices (CONCENTRATION=1)
[{'z': 269, 'y': 721, 'x': 66, 'confidence': 0.529296875}, {'z': 146, 'y': 251, 'x': 816, 'confidence': 0.349609375}]
     z    y    x  confidence
0  155  319  644    0.616211
     z    y    x  confidence
0  269  721   66    0.529297
1  146  251  816    0.349609
[     z    y    x  confidence
0  155  319  644    0.616211,      z    y   x  confidence
0  269  721  66    0.529297]


In [6]:
if fls.env=='local':
    ref_name = fls.code_dir + 'ref_yolo.pickle';
    assert str(res) == str(fls.dill_load(ref_name))

AssertionError: 

In [9]:
fls.dill_save(ref_name, str(res))

In [8]:
fls.dill_load(ref_name)

[     z    y    x
 0  155  319  644,
      z    y   x
 0  269  721  66]

In [18]:
model.preprocessor.return_uint8

False

In [8]:
model.step2Output

SelectSingleMotors(threshold=0.31931931931931934, vals=array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    0.049049,     0.05005,    0.051051,    0.052052,    0.053053,    0.054054,    0.055055,    0.056056,    0.057057,    0.058058,    0.059059,     0.06006,    0.061061,    0.062062,    0.063063,    0.064064,    0.065065,    0.066066,    0.067067,    0.068068,    0.069069,     0.07007,  